In [1]:
### import shutil
import tqdm
import numpy as np
import cv2
import os
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
import numpy as np
import os, sys
import pickle, functools, operator
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import joblib
from tensorflow.keras.utils import to_categorical
from keras.models import Model, load_model
from keras.layers import Input, LSTM, Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
import json
import random
from keras.utils.vis_utils import plot_model
import tensorflow as tf
import datetime

from pandas import read_csv
import numpy as np
from keras import Model
from keras.layers import Layer
import keras.backend as K
from keras.layers import Input, Dense, SimpleRNN , Concatenate
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.metrics import mean_squared_error

import random
random.seed(42)

In [2]:
class config():
    train_path = "./training_data"
    test_path = "./testing_data"
    batch_size = 160
    learning_rate = 0.0007
    epochs = 30
    latent_dim = 512
    num_encoder_tokens = 4096
    num_decoder_tokens = 1500
    time_steps_encoder = 80
    max_probability = -1
    validation_split = 0.15
    max_length = 30
    search_type = 'greedy'

In [3]:
config = config()

In [4]:
### import shutil
import tqdm
import numpy as np
import cv2
import os
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
import numpy as np
import os, sys
import pickle, functools, operator
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import joblib
from tensorflow.keras.utils import to_categorical
from keras.models import Model, load_model
from keras.layers import Input, LSTM, Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
import json
import random
from keras.utils.vis_utils import plot_model
import tensorflow as tf
import datetime

from pandas import read_csv
import numpy as np
from keras import Model
from keras.layers import Layer
import keras.backend as K
from keras.layers import Input, Dense, SimpleRNN , Concatenate
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.metrics import mean_squared_error

import random
random.seed(42)

## 02. Train & Test Dat Load

In [5]:
with open('./training_data/AllVideoDescriptions.txt') as f:
    lines = f.readlines()

In [6]:
with open('./train_split.txt','r') as f:
    train_l = f.readlines()
train_l = [l.replace('\n','') for l in train_l]

In [7]:
with open('./test_split.txt','r') as f:
    test_l = f.readlines()
test_l = [l.replace('\n','') for l in test_l]

In [8]:
with open('./val_split.txt','r') as f:
    val_l = f.readlines()
val_l = [l.replace('\n','') for l in val_l]

In [9]:
scripts = {}
for l in lines:
    id = l.split(' ')[0]
    script = ' '.join(l.split(' ')[1:])
    if id in scripts:
      scripts[id].append(script)
    else:
      scripts[id] = [script]

In [10]:
training_list = []
validation_list = []
test_list = [] 
vocab_list = []

for y in train_l:
  for caption in scripts[y]:
    caption = "<bos> " + caption + " <eos>"

    training_list.append([caption, y])

for y in val_l:
  for caption in scripts[y]:
    caption = "<bos> " + caption + " <eos>"

    validation_list.append([caption, y])
    
for y in test_l:
  for caption in scripts[y]:
    caption = "<bos> " + caption + " <eos>"

    test_list.append([caption, y])

In [11]:
train_list = training_list + validation_list + test_list

In [12]:
print(len(train_list))
random.shuffle(training_list)
random.shuffle(validation_list)
random.shuffle(test_list)

for train in training_list:
    vocab_list.append(train[0])
# Tokenizing the words
tokenizer = Tokenizer(num_words=1500)
tokenizer.fit_on_texts(vocab_list)
print(len(tokenizer.word_index))
x_data = {}
TRAIN_FEATURE_DIR = os.path.join('training_data', 'features_dir')
# Loading all the numpy arrays at once and saving them in a dictionary
for filename in os.listdir(TRAIN_FEATURE_DIR):
    f = np.load(os.path.join(TRAIN_FEATURE_DIR, filename))
    x_data[filename[:-4]] = f
print(len(training_list))
print(len(validation_list))
len(x_data)

80827
9463
48774
4290


1970

In [13]:
# Creating a custom data generator because we cannot load so many files at once
def load_datatest(train_path, epochs=100, x_data=x_data, tokenizer=tokenizer, num_decoder_tokens=1500,training_list=training_list, batch_size=32, maxlen=30):
    encoder_input_data = []
    decoder_input_data = []
    decoder_target_data = []
    videoId = []
    videoSeq = []
    # separating the videoId and the video captions
    for idx, cap in enumerate(training_list):
        caption = cap[0]
        videoId.append(cap[1])
        videoSeq.append(caption)
    # converting the captions to tokens and padding them to equal sizes
    train_sequences = tokenizer.texts_to_sequences(videoSeq)
    train_sequences = np.array(train_sequences)
    train_sequences = pad_sequences(train_sequences, padding='post',truncating='post', maxlen=maxlen)
    max_seq_length = train_sequences.shape[1]
    filesize = len(train_sequences)
    vCount = 0
    n = 0
    for i in range(epochs):
      for idx in  range(0,filesize):
        n += 1
        try:
          encoder_input_data.append(x_data[videoId[idx]])
        except:
          continue
        y = to_categorical(train_sequences[idx], num_decoder_tokens)
        decoder_input_data.append(y[:-1])
        decoder_target_data.append(y[1:])
        if n == batch_size:
          encoder_input = np.array(encoder_input_data)
          decoder_input = np.array(decoder_input_data)
          decoder_target = np.array(decoder_target_data)
          encoder_input_data = []
          decoder_input_data = []
          decoder_target_data = []
          n = 0
          yield ([encoder_input, decoder_input], decoder_target)
            
# writing the train and validation generator
train = load_datatest(train_path='training_data',batch_size=320, training_list=training_list, x_data=x_data, epochs=150)
valid = load_datatest(train_path='training_data',batch_size=320, training_list=validation_list, x_data=x_data, epochs=150)
test = load_datatest(train_path='test_data',batch_size=320, training_list=test_list, x_data=x_data, epochs=150)

In [14]:
del x_data

# 03. Load Trained Model

In [15]:
class Encoder(tf.keras.Model):
    def __init__(self, hidden_dim,time_steps_encoder,num_encoder_tokens):
        super(Encoder, self).__init__()
        # Define the RNN layer, LSTM
        self.hidden_dim = hidden_dim
        self.lstm = tf.keras.layers.LSTM( 
            hidden_dim, return_sequences=True, return_state=True, input_shape=(time_steps_encoder,num_encoder_tokens))

    def call(self, input_sequence, states):
        # Call the LSTM unit
        output, state_h, state_c = self.lstm(input_sequence, initial_state=states)

        return output, state_h, state_c

    def init_states(self, batch_size):
        # Return a all 0s initial states
        return (tf.zeros([batch_size, self.hidden_dim]),
                tf.zeros([batch_size, self.hidden_dim]))

In [16]:
class LuongAttention(tf.keras.Model):
    def __init__(self, rnn_size, attention_func):
        super(LuongAttention, self).__init__()
        self.attention_func = attention_func

        if attention_func not in ['dot', 'general', 'concat']:
            raise ValueError(
                'Attention score must be either dot, general or concat.')

        if attention_func == 'general':
            # General score function
            self.wa = tf.keras.layers.Dense(rnn_size)
        elif attention_func == 'concat':
            # Concat score function
            self.wa = tf.keras.layers.Dense(rnn_size, activation='tanh')
            self.va = tf.keras.layers.Dense(1)

    def call(self, decoder_output, encoder_output):
        if self.attention_func == 'dot':
            # Dot score function: decoder_output (dot) encoder_output
            # decoder_output has shape: (batch_size, 1, rnn_size)
            # encoder_output has shape: (batch_size, max_len, rnn_size)
            # => score has shape: (batch_size, 1, max_len)
            score = tf.matmul(decoder_output, encoder_output, transpose_b=True) # (batch_size, 1, max_len)
        elif self.attention_func == 'general':
            # General score function: decoder_output (dot) (Wa (dot) encoder_output)
            # decoder_output has shape: (batch_size, 1, rnn_size)
            # encoder_output has shape: (batch_size, max_len, rnn_size)
            # => score has shape: (batch_size, 1, max_len)
            score = tf.matmul(decoder_output, self.wa(
                encoder_output), transpose_b=True) #(batch_size, 1, max_len)
        elif self.attention_func == 'concat':
            # Concat score function: va (dot) tanh(Wa (dot) concat(decoder_output + encoder_output))
            # Decoder output must be broadcasted to encoder output's shape first
            decoder_output = tf.tile(
                decoder_output, [1, encoder_output.shape[1], 1]) #shape (batch size, max len,hidden_dim)

            # Concat => Wa => va
            # (batch_size, max_len, 2 * rnn_size) => (batch_size, max_len, rnn_size) => (batch_size, max_len, 1)
            score = self.va(
                self.wa(tf.concat((decoder_output, encoder_output), axis=-1))) # (batch_size, max len, 1)

            # Transpose score vector to have the same shape as other two above
            # (batch_size, max_len, 1) => (batch_size, 1, max_len)
            score = tf.transpose(score, [0, 2, 1]) #(batch_size, 1, max_len)

        # alignment a_t = softmax(score)
        alignment = tf.keras.activations.softmax(score, axis=-1) #(batch_size, 1, max_len)
        
        # context vector c_t is the weighted average sum of encoder output
        context = tf.matmul(alignment, encoder_output) # (batch_size, 1, hidden_dim)

        return context, alignment

In [17]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size,hidden_dim,time_steps_encoder,num_encoder_tokens,attention_func):
        super(Decoder, self).__init__()
        self.hidden_dim = hidden_dim
        self.attention = LuongAttention(hidden_dim, attention_func)
        self.lstm = tf.keras.layers.LSTM(
            hidden_dim, return_sequences=True, return_state=True,input_shape=(None,num_encoder_tokens))
        self.wc = tf.keras.layers.Dense(hidden_dim, activation='tanh')
        self.ws = tf.keras.layers.Dense(vocab_size,activation='softmax')
        
    def call(self, input_sequence, state,encoder_output):
        # Call the LSTM unit
        lstm_out, state_h, state_c = self.lstm(input_sequence, state)
        context , alignment = self.attention(lstm_out,encoder_output)
        lstm_out = tf.concat(
        [tf.squeeze(context, 1), tf.squeeze(lstm_out, 1)], 1)
        lstm_out = self.wc(lstm_out)
        logits = self.ws(lstm_out)


        return logits, state_h, state_c, alignment

In [18]:
save_model_path = './model_att_general_2022-06-02 '
#Set the length of the input and output vocabulary
time_steps_encoder=80
num_encoder_tokens=4096
latent_dim=512
time_steps_decoder=10
num_decoder_tokens=1500
batch_size=320
HIDDEN_DIM =  latent_dim

for ei,di in train:
    print(ei[0],ei[1],di)
    break

encoder = Encoder(HIDDEN_DIM,time_steps_encoder,num_encoder_tokens)
output =encoder(ei[0],encoder.init_states(batch_size))
encoder.load_weights(os.path.join(save_model_path, 'encoder.h5'))

decoder = Decoder(num_decoder_tokens,HIDDEN_DIM,time_steps_decoder,num_encoder_tokens,'general')
de_input = np.zeros((320, 1, 1500))
de_idx = tokenizer.word_index['bos']
de_state_h, de_state_c = output[1:]
decoder(de_input, (de_state_h, de_state_c), output[0])
decoder.load_weights(os.path.join(save_model_path, 'decoder.h5'))

/tmp/ipykernel_2445259/2347660553.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_sequences = np.array(train_sequences)


[[[0.28630525 0.         0.         ... 0.         0.         5.2103844 ]
  [0.21659842 0.         0.         ... 0.         0.03303999 4.683123  ]
  [1.072902   0.         0.         ... 0.         0.         4.327787  ]
  ...
  [0.6158212  0.         0.         ... 0.         0.         6.1895328 ]
  [0.6505584  0.         0.         ... 0.         0.         6.0730615 ]
  [0.         0.         0.         ... 0.         0.         6.9953475 ]]

 [[0.         0.         0.         ... 0.         2.0637891  1.4911273 ]
  [0.         0.         0.         ... 0.         1.2037091  1.4529827 ]
  [0.         0.         0.         ... 0.         0.         1.7469425 ]
  ...
  [0.         0.         1.0136279  ... 0.         0.2805397  0.        ]
  [0.         0.         0.9953279  ... 0.0639399  0.2224734  0.        ]
  [0.         0.         0.77224773 ... 0.25638026 0.13217995 0.        ]]

 [[0.         0.         0.34780604 ... 0.         2.3444505  0.69620126]
  [0.         0.      

2022-06-09 12:49:01.700186: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-09 12:49:01.701070: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-09 12:49:01.707002: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-09 12:49:01.707576: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-09 12:49:01.708122: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from S

# 04. Inference

In [19]:
# class to perform inference on all test files and save as test_output.txt
class Video2Text(object):
    ''' Initialize the parameters for the model '''
    def __init__(self,encoder,decoder,tokenizer,save_model_path):
        self.latent_dim = 512
        self.num_encoder_tokens = 4096
        self.num_decoder_tokens = 1500
        self.time_steps_encoder = 80
        self.time_steps_decoder = None
        self.preload = True
        self.preload_data_path = 'preload_data'
        self.max_probability = -1

        # processed data
        self.encoder_input_data = []
        self.decoder_input_data = []
        self.decoder_target_data = []
        self.tokenizer = tokenizer

        # models
        self.encoder_model = None
        self.decoder_model = None
        self.inf_encoder_model = None
        self.inf_decoder_model = None
        self.save_model_path = save_model_path
        self.test_path = 'testing_data'
        self.inf_encoder_model = encoder
        self.decoder = decoder
        
    
    def decode_sequence2bs(self, input_seq):
        en_initial_states = self.inf_encoder_model.init_states(1)
        output, state_h,state_c = self.inf_encoder_model(input_seq,en_initial_states)
        target_seq = np.zeros((1, 1, self.num_decoder_tokens))
        target_seq[0, 0, self.tokenizer.word_index['bos']] = 1
        self.beam_search(target_seq, [state_h,state_c],[],[],[],0,output)
        return decode_seq, decode_att

    def beam_search(self, target_seq, states_value, prob,  path, att, lens,en_output):
        global decode_seq
        global decode_att
        node = 2
        
        output_tokens, de_state_h, de_state_c, alignment = self.decoder(
                    target_seq, states_value, en_output)
        output_tokens = tf.reshape(output_tokens,self.num_decoder_tokens)#.reshape((self.num_decoder_tokens))

        sampled_token_index = tf.argsort(output_tokens,-1).numpy()[-50:][::-1]       
        states_value = [de_state_h, de_state_c]
        for i in range(node):
            if sampled_token_index[i] == 0:
                sampled_char = ''
            else:
                sampled_char = list(self.tokenizer.word_index.keys())[list(self.tokenizer.word_index.values()).index(sampled_token_index[i])]
            MAX_LEN = 15
            if(sampled_char != 'eos' and lens <= MAX_LEN):
                p = output_tokens[sampled_token_index[i]]
                if(sampled_char == ''):
                    p = 1
                prob_new = list(prob)
                prob_new.append(p)
                path_new = list(path)
                path_new.append(sampled_char)
                att_new = list(att)
                att_new.append(alignment)
                target_seq = np.zeros((1, 1, self.num_decoder_tokens))
                target_seq[0, 0, sampled_token_index[i]] = 1.
                self.beam_search(target_seq, states_value, prob_new, path_new, att_new, lens+1,en_output)
            else:
                p = output_tokens[sampled_token_index[i]]
                prob_new = list(prob)
                prob_new.append(p)
                p = functools.reduce(operator.mul, prob_new, 1)
                if(p > self.max_probability):
                    decode_seq = path
                    decode_att = att
                    self.max_probability = p

    def decoded_sentence_tuning(self, decoded_sentence):
        decode_str = []
        filter_string = ['bos', 'eos']
        unigram = {}
        last_string = ""
        for idx2, c in enumerate(decoded_sentence):
            if c in unigram:
                unigram[c] += 1
            else:
                unigram[c] = 1
            if(last_string == c and idx2 > 0):
                continue
            if c in filter_string:
                continue
            if len(c) > 0:
                decode_str.append(c)
            if idx2 > 0:
                last_string = c
        return decode_str

    def get_test_data(self, path):
        X_test = []
        X_test_filename = []
#         with open (os.path.join(path, 'testing_id.txt')) as testing_file:
#             lines = testing_file.readlines()
#             for filename in lines:
#                 filename = filename.strip()
#                 f = np.load(os.path.join(path , 'feat', filename + '.npy'))
#                 X_test.append(f)
#                 X_test_filename.append(filename[:-4])
#             X_test = np.array(X_test)
        for filename in test_l:
            try:
                f = np.load(os.path.join(path , 'features_dir', filename + '.npy'))
            except: 
                print('not exsits')
                continue
            X_test.append(f)
            X_test_filename.append(filename)
        return X_test, X_test_filename

    def test(self,method='beam'):
        X_test, X_test_filename = self.get_test_data(os.path.join(config.train_path)) 
        self.X_test , self.X_test_filename = X_test, X_test_filename
        # generate inference test outputs
        with open(os.path.join(self.save_model_path, f'test_output_{method}.txt'), 'w') as file:
            for idx, x in enumerate(X_test): 
                if method =='beam':
                    file.write(X_test_filename[idx]+',')
                    decoded_sentence,_ = self.decode_sequence2bs(x.reshape(-1, 80, 4096))
                    decode_str = self.decoded_sentence_tuning(decoded_sentence)
                    for d in decode_str:
                        file.write(d + ' ')
                elif method =='topk':
                    file.write(X_test_filename[idx]+',')
                    decoded_sentence = self.top_k_sampling(x.reshape(-1, 80, 4096))
                    file.write(decoded_sentence)
                file.write('\n')
                # re-init max prob
                self.max_probability = -1
                
    def index_to_word(self):
        # inverts word tokenizer
        index_to_word = {value: key for key, value in self.tokenizer.word_index.items()}
        return index_to_word
                
    def greedy_search(self, f):
        """
                :param f: the loaded numpy array after creating videos to frames and extracting features
                :return: the final sentence which has been predicted greedily
                """
        inv_map = self.index_to_word()
        en_initial_states = self.inf_encoder_model.init_states(1)
        output, state_h,state_c = self.inf_encoder_model(f.reshape(-1, 80, 4096),en_initial_states)
        target_seq = np.zeros((1, 1, 1500))
        sentence = ''
        target_seq[0, 0, self.tokenizer.word_index['bos']] = 1
        for i in range(15):
            output_tokens, state_h, state_c, alignment = self.decoder(target_seq, [state_h,state_c],output)
            output_tokens = tf.reshape(output_tokens,self.num_decoder_tokens)#.reshape((self.num_decoder_tokens))
            y_hat = np.argmax(output_tokens)
            if y_hat == 0:
                continue
            if inv_map[y_hat] is None:
                break
            else:
                sentence = sentence + inv_map[y_hat] + ' '
                target_seq = np.zeros((1, 1, 1500))
                target_seq[0, 0, y_hat] = 1
        return ' '.join(sentence.split()[:-1])
    
    def top_k_sampling(self,f):
        top_k = 2
        en_initial_states = self.inf_encoder_model.init_states(1)
        output, de_state_h,de_state_c = self.inf_encoder_model(f.reshape(-1, 80, 4096),en_initial_states)
        target_seq = np.zeros((1, 1, 1500))
        sentence = ''
        target_seq[0, 0, self.tokenizer.word_index['bos']] = 1

        for i in range(config.max_length):
            output_tokens, de_state_h, de_state_c, alignment = self.decoder(target_seq, [de_state_h,de_state_c],output)
            output_tokens = tf.reshape(output_tokens,self.num_decoder_tokens)#.reshape((self.num_decoder_tokens))
            top_index = np.argsort(output_tokens)[-top_k:]
            top_prob = np.sort(output_tokens)[-top_k:]
            y_hat =random.choices(top_index, weights=top_prob, k=1)[0]
            if y_hat == 0:
                continue
            if self.tokenizer.index_word[y_hat] is None:
                break
            else:
                sentence = sentence + self.tokenizer.index_word[y_hat] + ' '
                if self.tokenizer.index_word[y_hat] =='eos':
                    break
                target_seq = np.zeros((1, 1, 1500))
                target_seq[0, 0, y_hat] = 1
        return ' '.join(sentence.split()[:-1])

In [20]:
c = Video2Text(encoder,decoder,tokenizer,save_model_path)
#c.test('beam')

In [21]:
c.test('topk')